In [1]:
#======================================================================================================
# sentence-bert STS 데이터셋을 가지고, 훈련 및 평가 예시
#
# => 기존 (distil)bert 모델을 가지고, STS 훈련 및 평가 후, S-BERT로 만드는 예시임.

#=> 필요에 따라 출력 dimension을 768보다 작게 줄이고 싶을때 dense 모델을 추가해서 줄일수 있음
#=> reduce_out_dimension = True 로 하면, 출력 임베딩 dimension이 줄어들게 설정가능함

# => sentence-transformers 패키지를 이용하여 구현 함.(*pip install -U sentence-transformers 설치 필요)
#
# **learning rate는 기본이 2e-5임
#
# 도큐먼트 : https://www.sbert.net/index.html
# 소스참고 : https://github.com/BM-K/KoSentenceBERT-ETRI
#
# pip install -U sentence-transformers
#
#======================================================================================================
import torch.nn as nn
from torch.utils.data import DataLoader
import math
from sentence_transformers import models, losses
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import sys
import os
import gzip
import csv

sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging

logger = mlogging(loggername="s-bert-sts", logfilename="../../log/s-bert-sts")
device = GPU_info()
seed=111
seed_everything(seed)

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


logfilepath:../../log/s-bert-sts_2022-10-22.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [2]:
import os

# s-bert로 만들 원본 bert 경로
model_path = "../../data11/model/distilbert/bert-re-moco-corpus2-mecab/"
#model_path = "bongsoo/mdistilbertV2.1"

# 원본 bert를 sentencebert로 만든후 만들어진 s-bert 저장 경로
# => **해당 경로\eval 폴더에 similarity_evaluation_sts-dev_result.csv 파일로 각 epoch 마다 평가된 결과가 기록된다.
#smodel_path = 'output/training_nli_'+model_name.replace("/", "-")+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
smodel_path = "../../data11/model/sbert/bert-re-moco-corpus2-mecab-sts/"


use_korsts = 1     # 한국어 korsts 파일 (tsv 5,749개)
use_kluests = 1    # 한국어 kluests_v1.1 파일 (json 11,668개)
use_sts17 = 1      # 한국어 sts17-crosslingual-sts (jsonl 2,846개)
use_glue_sts = 1   # 영어 glue_sts (load_dataset 5,749개)
use_en_sts = 1     # 영어 stsb_multi_mt(load_dataset 15,676개) = stsb_multi_mt(5,749개) + mteb/sickr-sts(9,927개)

# KorSTS 학습, 평가 파일들
train_korsts_file = '../../data11/korpora/korsts/tune_train.tsv'
eval_korsts_file = '../../data11/korpora/korsts/tune_dev.tsv'

# KlueSTS 학습, 평가 파일들
train_kluests_file = '../../data11/korpora/klue-sts/klue-sts-v1.1_train.json'
eval_kluests_file = '../../data11/korpora/klue-sts/klue-sts-v1.1_dev.json'

# sts17-crosslingual-sts  학습 파일(*평가파일 없음)
train_sts17_file = '../../data11/korpora/sts17-crosslingual-sts/ko-ko.jsonl'

train_batch_size = 128
eval_batch_size = 64
num_epochs = 5      # 128 정도 해도 최상의 모델을 찾을수 있음 (*sbert는 eval이 최상인 모델이 out모델로 저장됨)
#num_epochs = 800
max_seq_length = 72
lr = 1e-4            # default=2e-5
eps = 1e-6
#============================================================================
# *출력 dimension을 줄일 경우에는 True로 하고, out_dimension에 줄일 값을 설정함
reduce_out_dimension = False  # True이면 dimension을 줄임=>Dense 모델 추가됨
out_dimension = 128
#============================================================================

# 모델과 tokenizer 를 불러옴
# => **사전파일(vocab.txt, *.json) 와 model 경로(config.json, pytorch_model.bin)가 같은 경로에 있어야 함.
word_embedding_model = models.Transformer(model_path, max_seq_length=max_seq_length, do_lower_case=False)

# word embedding_model 출력 
print(word_embedding_model)

Some weights of the model checkpoint at ../../data11/model/distilbert/bert-re-moco-corpus2-mecab/ were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Transformer({'max_seq_length': 72, 'do_lower_case': False}) with Transformer model: DistilBertModel 


In [3]:
# 2 bert 모델의 임베딩 풀링 정책을 설정(cls 이용, 워드임베딩 평균이용, 워드임베딩 max 이용)
# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),  #모델이 dimension(768)
                               pooling_mode_mean_tokens=True,  # 워드 임베딩 평균을 이용
                               pooling_mode_cls_token=False,   # cls 를 이용
                               pooling_mode_max_tokens=False)  # 워드 임베딩 값중 max 값을 이용
# pooling model 출력 
print(pooling_model)
print(pooling_model.get_sentence_embedding_dimension())

Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
768


In [4]:
# 3. dense 모델 추가(옵션)
#=> 필요에 따라 출력 dimension을 768보다 작게 줄이고 싶을때 dense 모델을 추가해서 줄임.
#=> https://www.sbert.net/docs/training/overview.html?highlight=dense 참조
if reduce_out_dimension:
    dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), # 입력 dimension은 앞에 pooling모델 embedding dimension으로 지정
                               out_features=out_dimension,  # 출력 dimension
                               activation_function=nn.Tanh())  # activation function은 Tahn으로 정의

In [5]:
# SBERT 모델 생성
if reduce_out_dimension:
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
else:
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 72, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


In [6]:
import json
from datasets import load_dataset

train_samples = []

####################################################################################################
# KorSTS 훈련 데이터 셋 설정(.tsv 파일)
####################################################################################################
if use_korsts == True:
    count = 0
    logger.info(f"Read STS train dataset=>{train_korsts_file}")
    with open(train_korsts_file, "rt", encoding="utf-8") as f:
        lines = f.readlines()
        for line in lines:
            text_a, text_b, score = line.split('\t')
            score = score.strip()
            score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
            
            if count < 3:
                print(f"{text_a}, {text_b}, {score}")
                
            train_samples.append(InputExample(texts= [text_a,text_b], label=score))
            count += 1
    logger.info(f'*{train_korsts_file} count: {count}')
####################################################################################################

####################################################################################################
# klue 훈련 데이터 셋 설정(.json 파일)
# => 아래처럼 load_dataset으로 불러와서 사용할수도 있음.
# datas = load_dataset("klue", "sts", split="train")
# for data in datas:
#        text_a = data["sentence1"]
#        text_b = data["sentence2"]
#        score = data["labels"]["label"]
#        score = float(score) / 5.0  
###################################################################################################           
if use_kluests == True:  
    count = 0
    logger.info(f"Read STS train dataset=>{train_kluests_file}")
    with open(train_kluests_file, "rt", encoding="utf-8") as f:
        datas = json.load(f)
        for data in datas:
            text_a = data["sentence1"]
            text_b = data["sentence2"]
            score = data["labels"]["label"]
            score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함

            if count < 3:
                print(f"{text_a}, {text_b}, {score}")

            train_samples.append(InputExample(texts= [text_a,text_b], label=score))
            count += 1
    logger.info(f'*{train_kluests_file} len: {count}')
####################################################################################################
# 한국어 sts17-crosslingual-sts 훈련 데이터셋 설정
# => jsonl : 여러개의 json 형식 파일이 각 줄마다 기록되어 있는 형태 파일
# => 패키지 설치 : !pip install jsonlines
####################################################################################################
if use_sts17 == True:
    import jsonlines
    count = 0
    logger.info(f"Read STS train dataset=>{train_sts17_file}")
    with jsonlines.open(train_sts17_file, "r") as f:
        for line in f:
            text_a = line["sentence1"]
            text_a = line["sentence2"]
            score = line["score"]
            score = float(score) / 5.0
            
            if count < 3:
                print(f"{text_a}, {text_b}, {score}")
                
            train_samples.append(InputExample(texts= [text_a,text_b], label=score))
            count += 1
            
    logger.info(f'*{train_kluests_file} len: {count}')
####################################################################################################

#############################################################################################
# 영문 sts 데이터셋 설정 (load_dataset)
# => stsb_multi_mt , mteb/sickr-sts 영문 sts 훈련 데이터 셋 불러오기
#############################################################################################
if use_en_sts == True:
    count = 0
    en_sts_dataset = load_dataset("stsb_multi_mt", name="en", split="train")
    for data in en_sts_dataset:
        text_a = data["sentence1"]
        text_b = data["sentence2"]
        score = data["similarity_score"]
        score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함

        if count < 3:
            print(f"{text_a}, {text_b}, {score}")

        train_samples.append(InputExample(texts= [text_a,text_b], label=score))
        count += 1
    logger.info(f'*stsb_multi_mt_en len: {count}')
    
    # mteb/sickr-sts 훈련데이터 불러옴
    count = 0    
    en_sts_dataset = load_dataset("mteb/sickr-sts", split="test")
    for data in en_sts_dataset:
        text_a = data["sentence1"]
        text_b = data["sentence2"]
        score = data["score"]
        score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함

        if count < 3:
            print(f"{text_a}, {text_b}, {score}")

        train_samples.append(InputExample(texts= [text_a,text_b], label=score))
        count += 1
    logger.info(f'*mteb/sickr-sts len: {count}')
#############################################################################################           
 
#############################################################################################
# GLUE STS 훈련 데이터셋 설정 (load_dataset)
#############################################################################################
if use_glue_sts == True:
    # glue stsb 훈련데이터 불러옴(5,749개)
    count = 0    
    en_sts_dataset = load_dataset("glue","stsb", split="train")
    for data in en_sts_dataset:
        text_a = data["sentence1"]
        text_b = data["sentence2"]
        score = data["label"]
        score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함

        if count < 3:
            print(f"{text_a}, {text_b}, {score}")

        train_samples.append(InputExample(texts= [text_a,text_b], label=score))
        count += 1
    logger.info(f'*glue_stsb len: {count}')
#############################################################################################

logger.info(f'------------------------------------------------------------------------')        
logger.info(f'*train_samples_len:{len(train_samples)}')
print(train_samples[0:3])

2022-10-23 06:32:05,782 - s-bert-sts - INFO - Read STS train dataset=>../../data11/korpora/korsts/tune_train.tsv


비행기가 이륙하고 있다., 비행기가 이륙하고 있다., 1.0
한 남자가 큰 플루트를 연주하고 있다., 남자가 플루트를 연주하고 있다., 0.76
한 남자가 피자에 치즈를 뿌려놓고 있다., 한 남자가 구운 피자에 치즈 조각을 뿌려놓고 있다., 0.76


2022-10-23 06:32:05,798 - s-bert-sts - INFO - *../../data11/korpora/korsts/tune_train.tsv count: 5749
2022-10-23 06:32:05,800 - s-bert-sts - INFO - Read STS train dataset=>../../data11/korpora/klue-sts/klue-sts-v1.1_train.json
2022-10-23 06:32:06,087 - s-bert-sts - INFO - *../../data11/korpora/klue-sts/klue-sts-v1.1_train.json len: 11668
2022-10-23 06:32:06,092 - s-bert-sts - INFO - Read STS train dataset=>../../data11/korpora/sts17-crosslingual-sts/ko-ko.jsonl
2022-10-23 06:32:06,109 - s-bert-sts - INFO - *../../data11/korpora/klue-sts/klue-sts-v1.1_train.json len: 2846


숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다., 숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다., 0.74
위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다., 시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다., 0.0
회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘., 사람들이 주로 네이버 메일을 쓰는 이유를 알려줘, 0.06
안전모를 쓴 한 남자가 춤을 추고 있다., 학회 홍보 메일은 회신 하지마, 1.0
아이가 말을 타고 있다., 학회 홍보 메일은 회신 하지마, 0.95
남자가 뱀에게 쥐를 먹이고 있다., 학회 홍보 메일은 회신 하지마, 1.0


Reusing dataset stsb_multi_mt (/MOCOMSYS/.cache/huggingface/datasets/stsb_multi_mt/en/1.0.0/a5d260e4b7aa82d1ab7379523a005a366d9b124c76a5a5cf0c4c5365458b0ba9)


...

A group of kids is playing in a yard and an old man is standing in the background, A group of boys in a yard is playing and a man is standing in the background, 0.9
A group of children is playing in the house and there is no man standing in the background, A group of kids is playing in a yard and an old man is standing in the background, 0.64
The young boys are playing outdoors and the man is smiling nearby, The kids are playing outdoors near a man with a smile, 0.9400000000000001


2022-10-23 06:32:12,484 - s-bert-sts - INFO - *mteb/sickr-sts len: 9927
Reusing dataset glue (/MOCOMSYS/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


A plane is taking off., An air plane is taking off., 1.0
A man is playing a large flute., A man is playing a flute., 0.7599999904632568
A man is spreading shreded cheese on a pizza., A man is spreading shredded cheese on an uncooked pizza., 0.7599999904632568


2022-10-23 06:32:14,568 - s-bert-sts - INFO - *glue_stsb len: 5749
2022-10-23 06:32:14,569 - s-bert-sts - INFO - ------------------------------------------------------------------------
2022-10-23 06:32:14,570 - s-bert-sts - INFO - *train_samples_len:41688


[<sentence_transformers.readers.InputExample.InputExample object at 0x7f26efedee80>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f26e6129a90>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f26e612f8b0>]


In [7]:
# 데이터 셋, 데이터 로더, 손실함수 정의

train_dataset = SentencesDataset(train_samples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)


In [8]:
#Read STSbenchmark dataset and use it as development set
# 평가데이터 불러오기
#korsts 파일로 두 문장간 유사도를 수치로(5.0이 만점=매우 유사) 측정함.
dev_samples = []

####################################################################################################
# KorSTS 평가 데이터 셋 설정(.tsv 파일)
####################################################################################################
if use_korsts == True:
    count = 0
    logger.info(f"Read STS dev dataset=>{eval_korsts_file}")
    with open(eval_korsts_file, 'rt', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            text_a, text_b, score = line.split('\t')
            score = score.strip()
            score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
            
            if count < 5:
                print(f"{text_a}, {text_b}, {score}")
            
            dev_samples.append(InputExample(texts= [text_a,text_b], label=score))
            count += 1
    logger.info(f'*{eval_korsts_file} len: {count}')
####################################################################################################  

####################################################################################################
# KlueSTS 평가 데이터 셋 설정(.json 파일)
# => 아래처럼 load_dataset으로 불러와서 사용할수도 있음.
# datas = load_dataset("klue", "sts", split="test")
# for data in datas:
#        text_a = data["sentence1"]
#        text_b = data["sentence2"]
#        score = data["labels"]["label"]
#        score = float(score) / 5.0  
####################################################################################################           
if use_kluests == True:
    count = 0
    logger.info(f"Read STS dev dataset=>{eval_kluests_file}")
    with open(eval_kluests_file, "rt", encoding="utf-8") as f:
        datas = json.load(f)
        for data in datas:
            text_a = data["sentence1"]
            text_b = data["sentence2"]
            score = data["labels"]["label"]
            score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함

            if count < 5:
                print(f"{text_a}, {text_b}, {score}")

            dev_samples.append(InputExample(texts= [text_a,text_b], label=score))
            count += 1
    logger.info(f'*{eval_kluests_file} len: {count}')
####################################################################################################  

####################################################################################################
# 영문 stsb_multi_mt 데이터 셋 설정(load_dataset)
####################################################################################################                
# stsb_multi_mt 영문 sts dev 데이터 셋 불러오기
if use_en_sts == True:
    count = 0
    en_sts_dataset = load_dataset("stsb_multi_mt", name="en", split="dev")
    for data in en_sts_dataset:
        text_a = data["sentence1"]
        text_b = data["sentence2"]
        score = data["similarity_score"]
        score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함

        if count < 3:
            print(f"{text_a}, {text_b}, {score}")

        dev_samples.append(InputExample(texts= [text_a,text_b], label=score))
        count += 1
    logger.info(f'*stsb_multi_mt len: {count}')
####################################################################################################  

####################################################################################################
# 영문 GLUE 데이터 셋 설정(load_dataset)
####################################################################################################  
if use_glue_sts == True:
    count = 0
    glue_stsb_dataset = load_dataset("glue","stsb", split="validation")
    for data in glue_stsb_dataset:
        text_a = data["sentence1"]
        text_b = data["sentence2"]
        score = data["label"]
        score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
        
        if count < 3:
            print(f"{text_a}, {text_b}, {score}")
            
        dev_samples.append(InputExample(texts= [text_a,text_b], label=score))
        count += 1
    logger.info(f'*glue-stsb len: {count}')
####################################################################################################  

logger.info(f'------------------------------------------------------------------------')        
logger.info(f'*dev_samples_len:{len(dev_samples)}')
print(dev_samples[0:3])

# 2개의 bert 모델에서 구한 2개의 embedding 값들의 cosine 유사도를 구해서, 이를 실제 score와 비교해서 유사도 측정함
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, 
                                                                 batch_size=eval_batch_size, 
                                                                 name='sts-dev')

2022-10-23 06:32:14,591 - s-bert-sts - INFO - Read STS dev dataset=>../../data11/korpora/korsts/tune_dev.tsv
2022-10-23 06:32:14,597 - s-bert-sts - INFO - *../../data11/korpora/korsts/tune_dev.tsv len: 1500
2022-10-23 06:32:14,598 - s-bert-sts - INFO - Read STS dev dataset=>../../data11/korpora/klue-sts/klue-sts-v1.1_dev.json
2022-10-23 06:32:14,618 - s-bert-sts - INFO - *../../data11/korpora/klue-sts/klue-sts-v1.1_dev.json len: 519


안전모를 가진 한 남자가 춤을 추고 있다., 안전모를 쓴 한 남자가 춤을 추고 있다., 1.0
어린아이가 말을 타고 있다., 아이가 말을 타고 있다., 0.95
한 남자가 뱀에게 쥐를 먹이고 있다., 남자가 뱀에게 쥐를 먹이고 있다., 1.0
한 여성이 기타를 연주하고 있다., 한 남자가 기타를 치고 있다., 0.48
한 여성이 플루트를 연주하고 있다., 남자가 플루트를 연주하고 있다., 0.55
무엇보다도 호스트분들이 너무 친절하셨습니다., 무엇보다도, 호스트들은 매우 친절했습니다., 0.9800000000000001
주요 관광지 모두 걸어서 이동가능합니다., 위치는 피렌체 중심가까지 걸어서 이동 가능합니다., 0.27999999999999997
학생들의 균형 있는 영어능력을 향상시킬 수 있는 학교 수업을 유도하기 위해 2018학년도 수능부터 도입된 영어 영역 절대평가는 올해도 유지한다., 영어 영역의 경우 학생들이 한글 해석본을 암기하는 문제를 해소하기 위해 2016학년도부터 적용했던 EBS 연계 방식을 올해도 유지한다., 0.26
다만, 도로와 인접해서 거리의 소음이 들려요., 하지만, 길과 가깝기 때문에 거리의 소음을 들을 수 있습니다., 0.74
형이 다시 캐나다 들어가야 하니 가족모임 일정은 바꾸지 마세요., 가족 모임 일정은 바꾸지 말도록 하십시오., 0.5


Reusing dataset stsb_multi_mt (/MOCOMSYS/.cache/huggingface/datasets/stsb_multi_mt/en/1.0.0/a5d260e4b7aa82d1ab7379523a005a366d9b124c76a5a5cf0c4c5365458b0ba9)
2022-10-23 06:32:15,765 - s-bert-sts - INFO - *stsb_multi_mt len: 1500


A man with a hard hat is dancing., A man wearing a hard hat is dancing., 1.0
A young child is riding a horse., A child is riding a horse., 0.95
A man is feeding a mouse to a snake., The man is feeding a mouse to the snake., 1.0


Reusing dataset glue (/MOCOMSYS/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
2022-10-23 06:32:16,932 - s-bert-sts - INFO - *glue-stsb len: 1500
2022-10-23 06:32:16,934 - s-bert-sts - INFO - ------------------------------------------------------------------------
2022-10-23 06:32:16,935 - s-bert-sts - INFO - *dev_samples_len:5019


A man with a hard hat is dancing., A man wearing a hard hat is dancing., 1.0
A young child is riding a horse., A child is riding a horse., 0.95
A man is feeding a mouse to a snake., The man is feeding a mouse to the snake., 1.0
[<sentence_transformers.readers.InputExample.InputExample object at 0x7f26efeb4f10>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f26efeb18e0>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f26dc177e20>]


In [9]:
#warmup_step은 10% 로 설정
warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) 
#warmup_steps = 0 # kcbert-config 참조함 

# evaluation_steps은 20%로 설정
evaluation_steps = int(len(train_dataset) * num_epochs / train_batch_size * 0.2)

logger.info(f"*IN-model:{model_path}")
logger.info(f"*OUT-model:{smodel_path}")
logger.info("*seed:{}, train_batch:{}, eval_batch:{}, epoch:{}, lr:{}, eps:{}, max_seq_length:{}, train_dataset:{}, Warmup-steps: {}, evaluation_step: {}".format(seed, train_batch_size, eval_batch_size, num_epochs, lr, eps, max_seq_length, len(train_dataset), warmup_steps, evaluation_steps))

# Train the model
# => **learning rate는 기본이 2e-5임
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=dev_evaluator,
          epochs=num_epochs,
          evaluation_steps=evaluation_steps,
          warmup_steps=warmup_steps,
          optimizer_params= {'lr': lr, 'eps': eps, 'correct_bias': False},
          save_best_model=True, # **기본 = True : eval 가장 best 모델을 output_Path에 저장함
          output_path=smodel_path
          )


2022-10-23 06:32:16,946 - s-bert-sts - INFO - *IN-model:../../data11/model/distilbert/bert-re-moco-corpus2-mecab/
2022-10-23 06:32:16,948 - s-bert-sts - INFO - *OUT-model:../../data11/model/sbert/bert-re-moco-corpus2-mecab-sts/
2022-10-23 06:32:16,949 - s-bert-sts - INFO - *seed:111, train_batch:128, eval_batch:64, epoch:5, lr:0.0001, eps:1e-06, max_seq_length:72, train_dataset:41688, Warmup-steps: 163, evaluation_step: 325
/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/326 [00:00<?, ?it/s]

Iteration:   0%|          | 0/326 [00:00<?, ?it/s]

Iteration:   0%|          | 0/326 [00:00<?, ?it/s]

Iteration:   0%|          | 0/326 [00:00<?, ?it/s]

Iteration:   0%|          | 0/326 [00:00<?, ?it/s]

In [10]:
##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
# => 훈련되어서 저장된 s-bert 모델을 불러와서 성능 평가 해봄
##############################################################################
import time
from sentence_transformers.evaluation import SimilarityFunction

# 테스트 파일=KorSTS 테스트파일 경로 지정
test_file = '../../data11/korpora/korsts/tune_test.tsv'

# 평가시 cosine 유사도등 측정 결과값 파일 (similarity_evaluation_xxxx.xls) 저장될 경로
output_path = './sts-test2'
os.makedirs(output_path, exist_ok=True)

test_samples = []
with open(test_file, 'rt', encoding='utf-8') as fIn:
    lines = fIn.readlines()
    for line in lines:
        s1, s2, score = line.split('\t')
        score = score.strip()
        score = float(score) / 5.0
        test_samples.append(InputExample(texts=[s1,s2], label=score))

logger.info("\n")
logger.info("======================TEST===================")
logger.info("\n\n")
logger.info(f"model save path > {smodel_path}")
start = time.time()
model = SentenceTransformer(smodel_path)

# 유사도 측정방식(COSINE, EUCLIDEAN, MANHATTAN, DOT_PRODUCT 중 선택 , 모두 spearman 방식임)
# => None 이면 아래 값들중 MAX 값 추력함
#main_similarity = None
main_similarity = SimilarityFunction.COSINE
#main_similarity = SimilarityFunction.EUCLIDEAN
#main_similarity = SimilarityFunction.MANHATTAN
#main_similarity = SimilarityFunction.DOT_PRODUCT

logger.info(f"main_similarity: {main_similarity}")

test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, main_similarity=main_similarity, batch_size=eval_batch_size, name='sts-test', show_progress_bar=True)
result = test_evaluator(model, output_path=output_path)

logger.info(f"\n")
logger.info(f"model path: {smodel_path}")
logger.info(f'=== result: {result} ===')
logger.info(f'=== 처리시간: {time.time() - start:.3f} 초 ===')
logger.info("==============================================")
logger.info("\n")

2022-10-23 06:38:15,632 - s-bert-sts - INFO - 

2022-10-23 06:38:15,634 - s-bert-sts - INFO - ======================TEST===================
2022-10-23 06:38:15,635 - s-bert-sts - INFO - 


2022-10-23 06:38:15,636 - s-bert-sts - INFO - model save path > ../../data11/model/sbert/bert-re-moco-corpus2-mecab-sts/
2022-10-23 06:38:17,387 - s-bert-sts - INFO - main_similarity: SimilarityFunction.COSINE


Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

2022-10-23 06:38:18,143 - s-bert-sts - INFO - 

2022-10-23 06:38:18,144 - s-bert-sts - INFO - model path: ../../data11/model/sbert/bert-re-moco-corpus2-mecab-sts/
2022-10-23 06:38:18,145 - s-bert-sts - INFO - === result: 0.8098808749723254 ===
2022-10-23 06:38:18,145 - s-bert-sts - INFO - === 처리시간: 2.508 초 ===
2022-10-23 06:38:18,145 - s-bert-sts - INFO - ==============================================
2022-10-23 06:38:18,146 - s-bert-sts - INFO - 



In [11]:
# 마지막 model 저장
#output_path = "../../data11/model/sbert/sbert-mdistilbertV3.1-last"
#model.save(output_path)